<a href="https://colab.research.google.com/github/Sibgat-Ul/KD_Temp_Scheduler/blob/main/465_train_1_approach1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
ifigotin_imagenetmini_1000_path = kagglehub.dataset_download('ifigotin/imagenetmini-1000')

print('Data source import complete.')


Data source import complete.


In [2]:
!ls -a /root/.cache/kagglehub/datasets/ifigotin/imagenetmini-1000/versions/1/imagenet-mini

.  ..  train  val


In [2]:
import argparse
import os
import random
import shutil
import time
import warnings

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

# from tqdm import tqdm
from torchvision.io import read_image, ImageReadMode
from torchvision.datasets import ImageFolder
from torchvision import transforms

from torch.utils.data import DataLoader, Dataset

if torch.cuda.is_available():
    torch.set_default_device('cuda')

PATH = "/kaggle/input/imagenetmini-1000/imagenet-mini"

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import tqdm

In [4]:
# Get weights
weights101 = models.ResNet101_Weights.DEFAULT
transforms = weights101.transforms()

dataset_path = "/root/.cache/kagglehub/datasets/ifigotin/imagenetmini-1000/versions/1/imagenet-mini"

# Transform and load your dataset
train_dataset = datasets.ImageFolder(
    dataset_path + '/train',
    transform=transforms
)

# DataLoader for the validation set (adjust the path accordingly)
val_dataset = datasets.ImageFolder(
    dataset_path + '/val',
    transform=transforms
)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [6]:
# Define temperature and alpha for distillation
TEMPERATURE = 3.0
ALPHA = 0.95

# Load Teacher and Student Models
teacher_model = models.resnet101(
    weights=weights101
).to(device)

# print("======TEACHER======")
# print(teacher_model)

# student_model = models.resnet34(weights=None).to(device)

# print("======STUDENT======")
# print(student_model)

# Freeze teacher model parameters
# for param in teacher_model.parameters():
    # param.requires_grad = False

# Define loss functions
# criterion = nn.CrossEntropyLoss()
# distillation_loss = nn.KLDivLoss(reduction='batchmean')

# Define optimizer for the student model
# optimizer = optim.Adam(student_model.parameters(), lr=0.001)
# optimizer = optim.SGD(student_model.parameters(), lr=0.1, momentum=0.9, nesterov=True)


#### Approach-2

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np

import torchvision.models as models
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck

class ResnetDT(ResNet):
    def __init__(self, block_type="BasicBlock", init_xavier=False, *args, **kwargs):
        super(ResnetDT, self).__init__(*args, **kwargs)
        self.temperature = nn.Parameter(torch.tensor(random.uniform(8.0, 15.0)))

        if init_xavier == True:
            for m in self.modules():
                if isinstance(m, nn.Linear):
                    nn.init.xavier_normal_(m.weight, gain = 1)

    def get_temperature(self):
        # Ensure temperature stays positive
        return F.softplus(self.temperature)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)

        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        # x = torch.div(self.fc(x), self.t_student)
        return self.fc(x)


class TeacherTemperatureScheduler:
    def __init__(
        self,
        initial_temp,
        patience=5,
        temp_min=1.0,
        temp_max=6.0,
        temp_step=1.0,
        improvement_threshold=1e-3
    ):
        self.temperature = initial_temp
        self.patience = patience
        self.temp_min = temp_min
        self.temp_max = temp_max
        self.temp_step = temp_step
        self.improvement_threshold = improvement_threshold

        # Initialize tracking variables
        self.best_loss = float('inf')
        self.waiting = 0
        self.loss_history = []

    def step(self, current_loss):
        """
        Update temperature based on loss trend
        Returns: bool indicating if temperature was adjusted
        """
        self.loss_history.append(current_loss)
        temp_adjusted = False

        # Need at least patience + 1 losses to make a decision
        if len(self.loss_history) > self.patience:
            # Get the trend of last patience+1 losses
            recent_losses = self.loss_history[-self.patience-1:]
            loss_improved = recent_losses[-1] < (min(recent_losses[:-1]) - self.improvement_threshold)
            loss_worsened = recent_losses[-1] > (min(recent_losses[:-1]) + self.improvement_threshold)

            if loss_improved:
                # Reset waiting counter if loss improved significantly
                self.waiting = 0
                if current_loss < self.best_loss:
                    self.best_loss = current_loss
            else:
                self.waiting += 1

            # Adjust temperature if waited enough
            if self.waiting >= self.patience:
                if loss_worsened:
                    # If loss is getting worse, increase temperature
                    self.temperature = min(
                        self.temp_max,
                        self.temperature + self.temp_step
                    )

                else:
                    # If loss plateaued or slightly worse, decrease temperature
                    self.temperature = max(
                        self.temp_min,
                        self.temperature - self.temp_step
                    )
                self.waiting = 0
                temp_adjusted = True

        return temp_adjusted

class DistillationTrainer:
    def __init__(
        self,
        student_model,
        teacher_model,
        alpha=0.01,
        scheduler_patience=5,
        scheduler_min_temp=1.0,
        scheduler_max_temp=6.0
    ):
        self.student = student_model
        self.teacher = teacher_model
        self.teacher_temperature = random.uniform(4.0, 6.0)
        self.alpha = alpha

        # Initialize temperature scheduler
        self.temp_scheduler = TeacherTemperatureScheduler(
            initial_temp=self.teacher_temperature,
            patience=scheduler_patience,
            temp_min=scheduler_min_temp,
            temp_max=scheduler_max_temp
        )

        print(f"Initial Student Temperature: {self.student.get_temperature().item():.2f}")
        print(f"Initial Teacher Temperature: {self.teacher_temperature:.2f}")

    def accuracy(self, output, target, topk=(1,)):
        """
        Computes the accuracy over the top-k predictions for the specified values of k.

        Args:
            output (torch.Tensor): Logits or predicted outputs from the model (batch_size x num_classes).
            target (torch.Tensor): True labels (batch_size).
            topk (tuple of int): Specifies the top-k accuracies to be calculated.

        Returns:
            list: List of accuracies corresponding to each value in topk.
        """
        with torch.no_grad():
            maxk = max(topk)  # Maximum k value, e.g., top-5 if topk=(1,5)
            batch_size = target.size(0)

            # Get the top-k indices with highest probability/logits
            _, pred = output.topk(maxk, dim=1, largest=True, sorted=True)

            # Convert to shape (k, batch_size) and check if predictions are correct
            pred = pred.t()
            correct = pred.eq(target.view(1, -1).expand_as(pred))

            # Calculate accuracy for each k in topk
            res = []
            for k in topk:
                correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
                res.append(correct_k.mul_(100.0 / batch_size).item())

            return res
    def compute_loss(self, student_logits, teacher_logits, targets):
        student_temp = self.student.get_temperature()
        teacher_temp = self.teacher_temperature

        # Standard cross-entropy with hard targets
        hard_loss = F.cross_entropy(student_logits, targets)

        # Soft targets from teacher
        soft_targets = F.softmax(teacher_logits / teacher_temp, dim=1)

        # Student predictions with learned temperature
        soft_pred = F.log_softmax(student_logits / student_temp, dim=1)

        # KL divergence loss scaled by temperatures
        distill_loss = F.kl_div(
            soft_pred,
            soft_targets,
            reduction='batchmean'
        ) * (student_temp * teacher_temp)

        # Combined loss
        total_loss = (1 - self.alpha) * hard_loss + self.alpha * distill_loss
        return total_loss, hard_loss, distill_loss

    def train(self, train_loader, val_loader, epochs, learning_rate=1e-3):
        epoch_acc1 = 0.0
        epoch_acc5 = 0.0

        val_acc1 = 0.0
        val_acc5 = 0.0

        optimizer = torch.optim.SGD([
            {'params': [p for n, p in self.student.named_parameters() if 'temperature' not in n]},
            {'params': self.student.temperature, 'lr': learning_rate * 0.1}
          ],
          lr=learning_rate,
          momentum=0.9,
          weight_decay=0.0001
      )

        for epoch in range(epochs):
            # Training phase
            self.student.train()
            train_loss = 0.0
            for batch_idx, (data, targets) in enumerate(train_loader):
                data = data.to('cuda')
                targets = targets.to('cuda')

                student_logits = self.student(data)
                with torch.no_grad():
                    teacher_logits = self.teacher(data)

                total_loss, hard_loss, distill_loss = self.compute_loss(
                    student_logits, teacher_logits, targets
                )

                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()

                with torch.no_grad():
                    acc1, acc5 = self.accuracy(student_logits, targets, (1, 5))

                # Update running metrics
                batch_size = data.size(0)
                epoch_acc1 += acc1 * batch_size
                epoch_acc5 += acc5 * batch_size

                train_loss += total_loss.item()

                if batch_idx % 100 == 0:
                    print(f'Epoch {epoch}, Batch {batch_idx}:')
                    print(f'  Loss: {total_loss.item():.4f}')
                    print(f'  Student Temp: {self.student.get_temperature().item():.2f}')
                    print(f'  Teacher Temp: {self.teacher_temperature:.2f}')
                    print(f'  Accuracy@1: {acc1:.2f}%, Accuracy@5: {acc5:.2f}%\n')
                    print(f'  logits v target: { student_logits} v {targets}')

            avg_train_loss = train_loss / len(train_loader)
            epoch_acc1 /= len(train_loader.dataset)
            epoch_acc5 /= len(train_loader.dataset)

            # Validation phase
            self.student.eval()
            val_loss = 0.0
            with torch.no_grad():
                for data, targets in val_loader:
                    student_logits = self.student(data)
                    teacher_logits = self.teacher(data)
                    total_loss, _, _ = self.compute_loss(
                        student_logits, teacher_logits, targets
                    )
                    val_loss += total_loss.item()

                    acc1, acc5 = self.accuracy(student_logits, targets, (1, 5))
                    val_acc1 += acc1 * data.size(0)
                    val_acc5 += acc5 * data.size(0)

            avg_val_loss = val_loss / len(val_loader)
            val_acc1 /= len(val_loader.dataset)
            val_acc5 /= len(val_loader.dataset)

            # Update teacher temperature using scheduler
            temp_adjusted = self.temp_scheduler.step(avg_val_loss)
            if temp_adjusted:
                self.teacher_temperature = self.temp_scheduler.temperature

            # Print epoch summary
            print(f'\nEpoch {epoch} Summary:')
            print(f'  Train Loss: {avg_train_loss:.4f}')
            print(f'  Val Loss: {avg_val_loss:.4f}')
            print(f'  Student Temperature: {self.student.get_temperature().item():.2f}')
            print(f'  Teacher Temperature: {self.teacher_temperature:.2f}')
            print(f'  Val Accuracy@1: {val_acc1:.2f}%')
            print(f'  Val Accuracy@5: {val_acc5:.2f}%\n')
            if temp_adjusted:
                print('  Teacher temperature adjusted!')
            print(f'  Scheduler waiting counter: {self.temp_scheduler.waiting}\n')

In [8]:
student_model = ResnetDT(block_type="BasicBlock", init_xavier=True, block=BasicBlock, layers=[3, 4, 6, 3], num_classes=1000)

In [9]:
trainer = DistillationTrainer(student_model.to('cuda'), teacher_model)

# Train
# train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, generator=torch.Generator(device='cuda'))
# val_loader = DataLoader(val_dataset, batch_size=256, shuffle=True, generator=torch.Generator(device='cuda'))
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

trainer.train(train_loader, val_loader, epochs=10)

Initial Student Temperature: 11.68
Initial Teacher Temperature: 5.96
Epoch 0, Batch 0:
  Loss: 4.2670
  Student Temp: 11.68
  Teacher Temp: 5.96
  Accuracy@1: 0.00%
  Accuracy@5: 0.00%

Epoch 0, Batch 100:
  Loss: 4.0387
  Student Temp: 11.68
  Teacher Temp: 5.96
  Accuracy@1: 0.00%
  Accuracy@5: 0.00%


Epoch 0 Summary:
  Train Loss: 4.1071
  Val Loss: 4.0387
  Student Temperature: 11.68
  Teacher Temperature: 5.96
  Val Accuracy@1: 0.15%
  Val Accuracy@5: 0.74%

  Scheduler waiting counter: 0

Epoch 1, Batch 0:
  Loss: 4.0035
  Student Temp: 11.68
  Teacher Temp: 5.96
  Accuracy@1: 0.00%
  Accuracy@5: 0.00%

Epoch 1, Batch 100:
  Loss: 3.9166
  Student Temp: 11.67
  Teacher Temp: 5.96
  Accuracy@1: 0.00%
  Accuracy@5: 0.00%


Epoch 1 Summary:
  Train Loss: 3.9538
  Val Loss: 3.9262
  Student Temperature: 11.67
  Teacher Temperature: 5.96
  Val Accuracy@1: 0.15%
  Val Accuracy@5: 0.64%

  Scheduler waiting counter: 0

Epoch 2, Batch 0:
  Loss: 3.8817
  Student Temp: 11.67
  Teacher Te

In [22]:
torch.cuda.empty_cache()

In [21]:
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 6            |        cudaMalloc retries: 7         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   6783 MiB |  14820 MiB |  79762 MiB |  72979 MiB |\n|       from large pool |   6725 MiB |  14748 MiB |  78724 MiB |  71999 MiB |\n|       from small pool |     57 MiB |    219 MiB |   1037 MiB |    979 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   6783 MiB |  14820 MiB |  79762 MiB |  72979 MiB |\n|       from large pool |   6725 MiB |  14748 MiB |